# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных: <br>

Данные геологоразведки трёх регионов находятся в файлах: <br>
* /datasets/geo_data_0.csv, <br>
* /datasets/geo_data_1.csv, <br>
* /datasets/geo_data_2.csv. <br>
* id — уникальный идентификатор скважины; <br>
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы); <br>
* product — объём запасов в скважине (тыс. баррелей). <br>

## Загрузка и подготовка данных

Во-первых, сделаем import необходимых библиотек. 

In [1]:
from sklearn.linear_model import LinearRegression
from math import sqrt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

Далее - считаем данные.

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv', index_col = 'id')
data2 = pd.read_csv('/datasets/geo_data_1.csv', index_col = 'id')
data3 = pd.read_csv('/datasets/geo_data_2.csv', index_col = 'id')

Изучим их.

In [3]:
data1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
data2.head()

,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305


Далее идёт вывод из анализа pandas-profiling, к сожалению, стабилизировать его работу не смог, поэтому удалил соответствующие ячейки.<br>

Забавно, что во втором и третьем регионе f2 и product практически полностью скоррелированы.<br>

Также довольно любопытны значения f0, f1 и f2. Насколько я понимаю, эти признаки уже отмасштабированы. <br>

Напишем функцию и разделим признаки во всех датасетах.

In [5]:
def ft(data): 
    target = data['product']
    features = data.drop('product', axis = 1)
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state = 12345)
    return features_train, features_valid, target_train, target_valid

In [6]:
features_train1, features_valid1, target_train1, target_valid1 = ft(data1) 
features_train2, features_valid2, target_train2, target_valid2 = ft(data2)
features_train3, features_valid3, target_train3, target_valid3 = ft(data3) 

Проверим на всякий случай разбиение.

In [7]:
print("Обучающая выборка:", features_train1.shape, target_train1.shape)
print("Валидационная выборка:", features_valid1.shape, target_valid1.shape)

Обучающая выборка: (75000, 3) (75000,)
Валидационная выборка: (25000, 3) (25000,)


## Обучение и проверка модели

Ох, оказывается разбивать данные нужно было на этом шаге, но да ладно. Переходим сразу к модели и предсказаниям.

In [8]:
def models(features_train, target_train, features_valid):
    model = LinearRegression().fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    return model, predicted_valid

In [9]:
model1, predicted_valid1 = models(features_train1, target_train1, features_valid1)
model2, predicted_valid2 = models(features_train2, target_train2, features_valid2)
model3, predicted_valid3 = models(features_train3, target_train3, features_valid3)

Рассчитаем средний запас предсказанного сырья и RMSE модели.

In [10]:
summary = defaultdict(list) #словарь для итоговой таблицы

In [11]:
summary['mean'].append(predicted_valid1.mean())
summary['mean'].append(predicted_valid2.mean())
summary['mean'].append(predicted_valid3.mean())

In [12]:
summary['rmse'].append(sqrt(mean_squared_error(target_valid1, predicted_valid1)))
summary['rmse'].append(sqrt(mean_squared_error(target_valid2, predicted_valid2)))
summary['rmse'].append(sqrt(mean_squared_error(target_valid3, predicted_valid3)))

In [13]:
df_summary = pd.DataFrame(summary, index = ['region1', 'region2', 'region3'])
df_summary.head()

,mean,rmse
region1,92.592568,37.579422
region2,68.728547,0.893099
region3,94.965046,40.029709


## Подготовка к расчёту прибыли

Сохраним в отдельные переменные все ключевые значения для расчётов.

In [14]:
revenue_per_product = 450000  # 450 тысяч рублей минус НДС
budget = 10000000000 # бюджет на разработку региона
count = 200 # максимальное количество используемых скважин
max_count = 500 # исследуемое количество используемых скважин

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [15]:
product_req = (budget // revenue_per_product) + 1 # общее необходимое количество единиц продукции c региона
print("Необходимое количество продукции для покрытия затрат:", product_req)
product_per_point = product_req / count # средне необходимое количество единиц продукции cо скважины
print("Необходимое среднее количество продукта в скважине:", product_per_point)

Необходимое количество продукции для покрытия затрат: 22223
Необходимое среднее количество продукта в скважине: 111.115


## Расчёт прибыли и рисков 

Напишием функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
1) Выберем скважины с максимальными значениями предсказаний. <br>
2) Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям. <br>
3) Рассчитаем прибыль для полученного объёма сырья. <br>

In [16]:
def profit(probs, target, count):
    probs_sorted = probs.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (selected.sum() * revenue_per_product) - budget

Посчитаем риски и прибыль для каждого региона: <br>
Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. <br>

Для начала напишем функцию для bootstrap.

In [17]:
def boots(region, target):
    state = np.random.RandomState(12345)
    rev = []
    risque = 0
    region = pd.Series(region)
    for i in range(1000):
        region_subsample = region.sample(n = max_count, replace = True, random_state = state)
        rev.append(profit(region_subsample, target, count))
    return pd.Series(rev)

Посчитаем прибыль для всех трёх регионов.

In [18]:
rev1 = boots(predicted_valid1, target_valid1)
rev2 = boots(predicted_valid2, target_valid2)
rev3 = boots(predicted_valid3, target_valid3)

Посчитаем среднюю прибыль, 5% квантиль и риск убытков.

In [19]:
def stats(data):
    print("Средняя прибыль:", data.mean())
    print("Нижний квантиль:", data.quantile(0.025))
    print("Верхний квантиль:", data.quantile(0.975))
    print("Доля прибыльных выборок:", data[data > 0].count() / 1000)
    print("Риск:", (data.values < 0).mean() * 100, "процентов")

In [20]:
print('Статистика первого региона:')
stats(rev1)

Статистика первого региона:
Средняя прибыль: 396164984.8023711
Нижний квантиль: -111215545.89049526
Верхний квантиль: 909766941.5534226
Доля прибыльных выборок: 0.931
Риск: 6.9 процентов


In [21]:
print('Статистика второго региона:')
stats(rev2)

Статистика второго региона:
Средняя прибыль: 456045105.7866608
Нижний квантиль: 33820509.39898363
Верхний квантиль: 852289453.866036
Доля прибыльных выборок: 0.985
Риск: 1.5 процентов


In [22]:
print('Статистика третьего региона:')
stats(rev3)

Статистика третьего региона:
Средняя прибыль: 404403866.5683568
Нижний квантиль: -163350413.39560106
Верхний квантиль: 950359574.9237995
Доля прибыльных выборок: 0.924
Риск: 7.6 процентов


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

## Выводы

В результате выполнения проекта построена модель для определения региона с с максимальной прибылью в добывающей компании «ГлавРосГосНефть».<br>
Рассчитан средний запас предсказанного сырья и RMSE модели. <br>

In [23]:
df_summary.head()

,mean,rmse
region1,92.592568,37.579422
region2,68.728547,0.893099
region3,94.965046,40.029709


Рассчитан достаточный объём сырья для безубыточной разработки новой скважины. <br>
Необходимое количество продукции для покрытия затрат: 22223 <br>
Необходимое среднее количество продукта в скважине: 111.115

Посчитаны риски и прибыль для каждого региона. <br>
Наиболее стабильную и выгодную картину показал второй регион: <br>
Средняя прибыль: 456045105.7866609 <br>
Нижний квантиль: 33820509.39898549 <br>
Верхний квантиль: 852289453.866036 <br>
Доля прибыльных выборок 0.985 <br>

Верхний квантиль выше в первом и третьем регионах, но, к сожалению, доля убыточных выборок там выше.